In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk
import sklearn.linear_model as skl
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder, OneHotEncoder
from sklearn.metrics import make_scorer, mean_squared_error,roc_curve, auc, roc_auc_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split,StratifiedKFold,cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.linear_model import LogisticRegression, LinearRegression

import matplotlib.pyplot as plt
from IPython.display import display
from itertools import groupby

In [10]:
df = pd.read_csv("all_teams.csv")
df.head()

,team,season,name,gameId,playerTeam,opposingTeam,home_or_away,gameDate,position,situation,...,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame
0,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,other,...,1.0,1.000,0.0,0.017,0.000,0.000,0.037,0.037,0.037,0
1,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,all,...,31.0,30.369,5.0,0.396,0.168,0.168,2.917,2.833,2.714,0
2,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,5on5,...,20.0,19.369,3.0,0.237,0.168,0.168,1.862,1.777,1.665,0
3,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,4on5,...,9.0,9.000,1.0,0.124,0.000,0.000,0.795,0.795,0.789,0
4,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,5on4,...,1.0,1.000,1.0,0.019,0.000,0.000,0.224,0.224,0.224,0


In [11]:
#dont care about playoff games
df = df[df['playoffGame']==0]
#only want 2022-2023 season
df = df[df['season']>2021]
#for totals
df['totalGoals'] = df["goalsFor"]+df["goalsAgainst"]
#to see who won the game
df["gameWon"] = np.where((df["goalsFor"]>df["goalsAgainst"]), 1, 0)
df = df.reset_index(drop=True)

df.head()

,team,season,name,gameId,playerTeam,opposingTeam,home_or_away,gameDate,position,situation,...,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame,totalGoals,gameWon
0,NYR,2022,NYR,2022020003,NYR,TBL,HOME,20221011,Team Level,other,...,0.0,0.147,0.000,0.000,1.534,1.534,1.199,0,1.0,0
1,NYR,2022,NYR,2022020003,NYR,TBL,HOME,20221011,Team Level,all,...,5.0,0.465,0.176,0.176,3.043,3.102,2.754,0,4.0,1
2,NYR,2022,NYR,2022020003,NYR,TBL,HOME,20221011,Team Level,5on5,...,5.0,0.208,0.176,0.176,1.311,1.371,1.360,0,1.0,1
3,NYR,2022,NYR,2022020003,NYR,TBL,HOME,20221011,Team Level,4on5,...,0.0,0.109,0.000,0.000,0.198,0.198,0.195,0,1.0,1
4,NYR,2022,NYR,2022020003,NYR,TBL,HOME,20221011,Team Level,5on4,...,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0,1.0,1


In [12]:
c = df.corr()['totalGoals'].drop("totalGoals")
# print(c)
pos_c = c.sort_values(ascending=True) #[90:]
print(pos_c)

<ipython-input-12-0eb334f348e2>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  c = df.corr()['totalGoals'].drop("totalGoals")


gameId              -0.002670
gameDate            -0.000256
corsiPercentage      0.038251
xGoalsPercentage     0.045027
fenwickPercentage    0.049130
                       ...   
iceTime              0.830881
goalsAgainst         0.859370
goalsFor             0.859370
season                    NaN
playoffGame               NaN
Name: totalGoals, Length: 105, dtype: float64


In [20]:
df.dtypes

team                                          object
season                                         int64
name                                          object
gameId                                         int64
playerTeam                                    object
                                              ...   
scoreAdjustedTotalShotCreditAgainst          float64
scoreFlurryAdjustedTotalShotCreditAgainst    float64
playoffGame                                    int64
totalGoals                                   float64
gameWon                                        int64
Length: 113, dtype: object

In [19]:
t = df[1::5].groupby(['team']).mean()
t.sort_values(by=['totalGoals'])


<ipython-input-19-bfad426db393>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  t = df[1::5].groupby(['team']).mean()


,season,gameId,gameDate,xGoalsPercentage,corsiPercentage,fenwickPercentage,iceTime,xOnGoalFor,xGoalsFor,xReboundsFor,...,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame,totalGoals,gameWon
team,,,,,,,,,,,,,,,,,,,,,
MIN,2022.0,2.022021e+09,2.022501e+07,0.515352,0.506952,0.515068,3668.206349,32.240651,3.164619,2.234889,...,3.984127,0.459698,0.572952,0.574476,2.875873,2.877825,2.777937,0.0,5.396825,0.460317
NYI,2022.0,2.022021e+09,2.022502e+07,0.494962,0.487083,0.485697,3641.876923,30.625077,3.117508,2.192477,...,5.584615,0.515815,0.772831,0.772831,2.946815,2.950785,2.874831,0.0,5.538462,0.492308
NSH,2022.0,2.022020e+09,2.022490e+07,0.490137,0.486520,0.483713,3642.000000,31.344883,3.482600,2.300567,...,5.100000,0.540567,0.686600,0.686600,3.455767,3.457550,3.338367,0.0,5.783333,0.466667
BOS,2022.0,2.022021e+09,2.022492e+07,0.549344,0.513390,0.517927,3647.967742,32.609952,3.503790,2.350645,...,4.838710,0.476823,0.583887,0.586194,2.798855,2.773210,2.695694,0.0,5.870968,0.758065
WPG,2022.0,2.022021e+09,2.022486e+07,0.509173,0.507324,0.506103,3623.650794,31.580349,3.262270,2.230238,...,6.968254,0.473524,0.567492,0.567492,3.069476,3.073444,2.981603,0.0,5.888889,0.555556
DAL,2022.0,2.022020e+09,2.022472e+07,0.541448,0.515417,0.509306,3655.238095,32.571556,3.486762,2.252349,...,4.380952,0.472238,0.497635,0.497635,2.866635,2.854302,2.796302,0.0,5.888889,0.492063
VGK,2022.0,2.022020e+09,2.022448e+07,0.532950,0.496111,0.508213,3655.758065,31.967935,3.245742,2.280565,...,3.903226,0.464048,0.495355,0.504290,2.797226,2.796919,2.734194,0.0,5.903226,0.532258
PHI,2022.0,2.022020e+09,2.022477e+07,0.451482,0.468123,0.476124,3636.806452,29.272839,2.754419,2.011694,...,5.387097,0.506532,0.607355,0.607355,3.245548,3.261710,3.180887,0.0,5.903226,0.354839
CAR,2022.0,2.022020e+09,2.022474e+07,0.586740,0.595070,0.590075,3655.750000,36.952083,3.713400,2.657567,...,4.650000,0.416500,0.544650,0.545233,2.490150,2.467850,2.408867,0.0,5.933333,0.616667


In [14]:
#every 2nd row of a game has situation all, need to find icetime for that
icetime = []
for i in range(1,len(df),5): #have to skip 5 because of this dataset, and start at index 1 which is all situations
    icetime.append(df.iloc[i][2])
averageice = np.mean(icetime)
# print(icetime)
avgicemin = averageice/60
print("Connor has played {} minutes per game in the NHL".format(avgicemin.round(2)))


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U3'), dtype('<U3')) -> None

In [17]:
tst = df[1::5]
tst

,team,season,name,gameId,playerTeam,opposingTeam,home_or_away,gameDate,position,situation,...,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame,totalGoals,gameWon
1,NYR,2022,NYR,2022020003,NYR,TBL,HOME,20221011,Team Level,all,...,5.0,0.465,0.176,0.176,3.043,3.102,2.754,0,4.0,1
6,NYR,2022,NYR,2022020017,NYR,MIN,AWAY,20221013,Team Level,all,...,2.0,0.626,0.954,0.954,4.128,3.953,3.584,0,10.0,1
11,NYR,2022,NYR,2022020023,NYR,WPG,AWAY,20221014,Team Level,all,...,14.0,0.472,0.407,0.407,4.286,4.232,4.138,0,5.0,0
16,NYR,2022,NYR,2022020039,NYR,ANA,HOME,20221017,Team Level,all,...,7.0,0.256,0.216,0.216,1.668,1.671,1.669,0,10.0,1
21,NYR,2022,NYR,2022020064,NYR,SJS,HOME,20221020,Team Level,all,...,1.0,0.389,1.156,1.156,2.354,2.437,2.397,0,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9956,CAR,2022,CAR,2022020911,CAR,STL,HOME,20230221,Team Level,all,...,4.0,0.523,0.164,0.164,2.480,2.457,2.444,0,5.0,1
9961,CAR,2022,CAR,2022020933,CAR,OTT,HOME,20230224,Team Level,all,...,2.0,0.621,0.070,0.070,2.419,2.407,2.362,0,4.0,1
9966,CAR,2022,CAR,2022020938,CAR,ANA,HOME,20230225,Team Level,all,...,7.0,0.348,0.416,0.416,1.570,1.655,1.634,0,5.0,0
9971,CAR,2022,CAR,2022020972,CAR,VGK,AWAY,20230301,Team Level,all,...,3.0,0.337,0.301,0.301,3.287,3.244,3.234,0,5.0,0


In [19]:
t = df[df["gameId"]==2022020003]
t

,team,season,name,gameId,playerTeam,opposingTeam,home_or_away,gameDate,position,situation,...,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame
5380,NYR,2022,NYR,2022020003,NYR,TBL,HOME,20221011,Team Level,other,...,7.0,7.000,0.0,0.147,0.000,0.000,1.534,1.534,1.199,0
5381,NYR,2022,NYR,2022020003,NYR,TBL,HOME,20221011,Team Level,all,...,34.0,34.315,5.0,0.465,0.176,0.176,3.043,3.102,2.754,0
5382,NYR,2022,NYR,2022020003,NYR,TBL,HOME,20221011,Team Level,5on5,...,20.0,20.315,5.0,0.208,0.176,0.176,1.311,1.371,1.360,0
5383,NYR,2022,NYR,2022020003,NYR,TBL,HOME,20221011,Team Level,4on5,...,7.0,7.000,0.0,0.109,0.000,0.000,0.198,0.198,0.195,0
5384,NYR,2022,NYR,2022020003,NYR,TBL,HOME,20221011,Team Level,5on4,...,0.0,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0
124040,TBL,2022,TBL,2022020003,TBL,NYR,AWAY,20221011,Team Level,other,...,2.0,2.000,0.0,0.017,0.000,0.000,0.892,0.892,0.892,0
124041,TBL,2022,TBL,2022020003,TBL,NYR,AWAY,20221011,Team Level,all,...,49.0,48.988,5.0,0.688,1.358,1.358,4.919,4.868,4.578,0
124042,TBL,2022,TBL,2022020003,TBL,NYR,AWAY,20221011,Team Level,5on5,...,25.0,24.988,5.0,0.302,0.485,0.485,1.930,1.878,1.834,0
124043,TBL,2022,TBL,2022020003,TBL,NYR,AWAY,20221011,Team Level,4on5,...,17.0,17.000,0.0,0.329,0.873,0.873,1.685,1.685,1.451,0
124044,TBL,2022,TBL,2022020003,TBL,NYR,AWAY,20221011,Team Level,5on4,...,5.0,5.000,0.0,0.041,0.000,0.000,0.413,0.413,0.401,0


In [12]:
# print(df.columns.tolist())
cols = df.columns.tolist()
for c in cols:
    print(c)

team
season
name
gameId
playerTeam
opposingTeam
home_or_away
gameDate
position
situation
xGoalsPercentage
corsiPercentage
fenwickPercentage
iceTime
xOnGoalFor
xGoalsFor
xReboundsFor
xFreezeFor
xPlayStoppedFor
xPlayContinuedInZoneFor
xPlayContinuedOutsideZoneFor
flurryAdjustedxGoalsFor
scoreVenueAdjustedxGoalsFor
flurryScoreVenueAdjustedxGoalsFor
shotsOnGoalFor
missedShotsFor
blockedShotAttemptsFor
shotAttemptsFor
goalsFor
reboundsFor
reboundGoalsFor
freezeFor
playStoppedFor
playContinuedInZoneFor
playContinuedOutsideZoneFor
savedShotsOnGoalFor
savedUnblockedShotAttemptsFor
penaltiesFor
penalityMinutesFor
faceOffsWonFor
hitsFor
takeawaysFor
giveawaysFor
lowDangerShotsFor
mediumDangerShotsFor
highDangerShotsFor
lowDangerxGoalsFor
mediumDangerxGoalsFor
highDangerxGoalsFor
lowDangerGoalsFor
mediumDangerGoalsFor
highDangerGoalsFor
scoreAdjustedShotsAttemptsFor
unblockedShotAttemptsFor
scoreAdjustedUnblockedShotAttemptsFor
dZoneGiveawaysFor
xGoalsFromxReboundsOfShotsFor
xGoalsFromActualRebou

In [29]:
tdf = df.groupby("gameId")["team"]
tdf.head()

5380      NYR
5381      NYR
5382      NYR
5383      NYR
5384      NYR
         ... 
166920    VAN
166921    VAN
166922    VAN
166923    VAN
166924    VAN
Name: team, Length: 4990, dtype: object